In [46]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from datetime import datetime
import matplotlib.pyplot as plt

In [47]:
## Defining a  scripting function to read all the files from the dataset
def folder_files(folder):
    folder_path = os.path.expanduser(folder)
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    
    dataframes = []
    for file in csv_files:
        csv_path = os.path.join(folder_path, file)
        df = pd.read_csv(csv_path)
        dataframes.append(df)
        print(f" File '{file}' has been loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
    
    print(f"Total number of CSV files loaded: {len(dataframes)}")
    return dataframes

ecommerce_folder_files = folder_files(r'C:\Users\Lenovo\OneDrive\Desktop\Ecommerce Project')

start_time= datetime.now()

try:
    customers_dataset= ecommerce_folder_files[0]
    geolocation_dataset= ecommerce_folder_files[1]
    orders_dataset= ecommerce_folder_files[2]
    order_items_dataset= ecommerce_folder_files[3]
    order_payments_dataset= ecommerce_folder_files[4]
    order_reviews_dataset= ecommerce_folder_files[5]
    products_dataset= ecommerce_folder_files[6]
    product_category_name_translation= ecommerce_folder_files[7]
    sellers_dataset= ecommerce_folder_files[8]
    print("All the files have been successfully loaded in the notebook")

except IndexError as e_1:
    print("Error: Some files are missing. Check if all required CSVs are present in the folder.")
    print(f"Details: {e_1}")

except Exception as e_2:
    print("An unexpected error occurred while loading the datasets.")
    print(f"Details: {e_2}")

end_time= datetime.now()
total_time_taken= abs((start_time-end_time)).total_seconds()
print(f" The total time taken to load the datasets is {total_time_taken} seconds")

 File 'customers_dataset.csv' has been loaded successfully with 99441 rows and 5 columns.
 File 'geolocation_dataset.csv' has been loaded successfully with 1000163 rows and 5 columns.
 File 'orders_dataset.csv' has been loaded successfully with 99441 rows and 8 columns.
 File 'order_items_dataset.csv' has been loaded successfully with 112650 rows and 7 columns.
 File 'order_payments_dataset.csv' has been loaded successfully with 103886 rows and 5 columns.
 File 'order_reviews_dataset.csv' has been loaded successfully with 99224 rows and 7 columns.
 File 'products_dataset.csv' has been loaded successfully with 32951 rows and 9 columns.
 File 'product_category_name_translation.csv' has been loaded successfully with 71 rows and 2 columns.
 File 'sellers_dataset.csv' has been loaded successfully with 3095 rows and 4 columns.
Total number of CSV files loaded: 9
All the files have been successfully loaded in the notebook
 The total time taken to load the datasets is 0.979565 seconds


In [48]:
### Scripting For Data Cleaning and Data Pre-processing

############################ Cleaning Customers Dataset ############################################################################
def data_cleaning_customers(df):
    df['customer_city']= df['customer_city'].str.title()
    df[df.columns]= df.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)
    duplicate_records_customers = df[df.duplicated(subset=['customer_id'], keep=False)]
    df = df.drop_duplicates(subset=['customer_id'], keep='first')
    print(f" Total number of records in the customers dataset are {df.shape[0]}")
    print(f" The total number of customer ids in the customers dataset are {df['customer_id'].nunique()}")
    print(f" The total number of unique customer ids in the customers dataset are {df['customer_unique_id'].nunique()}")
    print(f" The total number of duplicates in the customers dataset are {df[df.duplicated()].size}")
    print(f" The primary key for the customers dataset is the column: {df.columns[0]} whereas the foreign key is {df.columns[1]}")
    return df,duplicate_records_customers

customers_dataset, duplicate_customers= data_cleaning_customers(customers_dataset)

 Total number of records in the customers dataset are 99441
 The total number of customer ids in the customers dataset are 99441
 The total number of unique customer ids in the customers dataset are 96096
 The total number of duplicates in the customers dataset are 0
 The primary key for the customers dataset is the column: customer_id whereas the foreign key is customer_unique_id


In [49]:

########################### Cleaning the Orders Dataset #######################################################################
def data_cleaning_orders(df1):
    df1[df1.columns]= df1.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)
    df1['order_purchase_timestamp']= pd.to_datetime(df1['order_purchase_timestamp'].str[0:10],format="%d-%m-%Y",dayfirst=True)
    df1['order_approved_at']= pd.to_datetime(df1['order_approved_at'].str[0:10],format="%d-%m-%Y",dayfirst=True)
    df1['order_delivered_carrier_date']= pd.to_datetime(df1['order_delivered_carrier_date'].str[0:10],format="%d-%m-%Y",dayfirst=True)
    df1['order_delivered_customer_date']= pd.to_datetime(df1['order_delivered_customer_date'].str[0:10],format="%d-%m-%Y",dayfirst=True)
    df1['order_estimated_delivery_date']= pd.to_datetime(df1['order_estimated_delivery_date'].str[0:10],format="%d-%m-%Y",dayfirst=True)
    df1['order_status']= df1['order_status'].str.title()
    duplicated_records_orders= df1[df1.duplicated(subset=['order_id','customer_id'], keep=False)]
    print(f" The total number of records in the orders_dataset are {df1.shape[0]}")
    print(f" The total number of duplicated records in the orders_dataset are {duplicated_records_orders.shape[0]}")
    print(f" The total number of unique order ids in the orders_dataset are {df1['order_id'].nunique()}")
    print(f" The total number of unique customer ids in the orders_dataset are {df1['customer_id'].nunique()}")
    print(" For the orders_dataset, the column order_id serves as the primary key whereas the customer id column serves as the foreign key")
    return df1,duplicated_records_orders

orders_dataset, duplicate_orders = data_cleaning_orders(orders_dataset)
orders_dataset.head(5)

#############################################################################################################################################
## The order_items_dataset contaisn the information about orders i.e. which products were included in that particular order, what was the shipping date 
## of the order, the order value as well as the freight value that is used in the transportation of the order. Hence we shall make aggregated table
## from this particular dataset and then join with the order_dataset to fidn the values corresponding to each order and then make an aggregated table
## for orders.

def data_cleaning_order_items_dataset(df2):
    df2['shipping_limit_date']= pd.to_datetime(df2['shipping_limit_date'].str[0:10],dayfirst=True)
    df2[df2.columns]= df2.apply(lambda col: col.str.strip() if col.dtype== 'object' else col)
    final_order_items_dataset= df2.groupby(['order_id']).agg(total_order_value=('price','sum'),total_freight_charges=('freight_value','sum')).reset_index().drop_duplicates()
    return final_order_items_dataset

aggregated_order_value= data_cleaning_order_items_dataset(order_items_dataset)
aggregated_order_value

##################################################################################################################################################
## We will be performing join between the orders_dataset and the aggregated_order_value dataset to fetch the order value and freight cost for each order
df3 = pd.merge(orders_dataset,aggregated_order_value, how='left',on='order_id')
df3['total_order_value']= np.where((df3['total_order_value'].isnull()) & (df3['order_status'].isin(['Unavailable','Canceled'])),0,df3['total_order_value'])

final_orders= df3

 The total number of records in the orders_dataset are 99441
 The total number of duplicated records in the orders_dataset are 0
 The total number of unique order ids in the orders_dataset are 99441
 The total number of unique customer ids in the orders_dataset are 99441
 For the orders_dataset, the column order_id serves as the primary key whereas the customer id column serves as the foreign key


In [50]:

########################### Cleaning the Order Reviews Dataset #######################################################################

def orders_reviews_cleaning(df4):
    df4[df4.columns]= df4.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)
    df4['review_creation_date']= pd.to_datetime(df4['review_creation_date'].str[0:10],format="%d-%m-%Y",dayfirst=True)
    df4['review_answer_timestamp']= pd.to_datetime( df4['review_answer_timestamp'].str[0:10],format="%d-%m-%Y",dayfirst=True)
    df4.drop(columns={'Column1','Column2'},inplace=True)
    duplicated_records_orders_reviews= df4[df4.duplicated(subset=['review_id','order_id'],keep=False)]
    df4= df4.drop_duplicates(subset=['review_id','order_id'], keep='first')
    print(f"The total number of duplicated records in the dataset are {duplicated_records_orders_reviews.shape[0]}")
    print(f"The total number of records after dropping the duplicates is {df4.shape[0]}")
    return df4, duplicated_records_orders_reviews

order_reviews_dataset, order_review_duplicated_records= orders_reviews_cleaning(order_reviews_dataset)

The total number of duplicated records in the dataset are 0
The total number of records after dropping the duplicates is 99224


In [51]:
########################### Cleaning the Orders Dataset #######################################################################
## We shall be combining two datasets:products_dataset and product_category_name_translation to fetch the product names and the product category transalation

def merging_product_info(df5,df6):
    merged_products_info= pd.merge(df5[['product_id','product_category_name']],df6,how='left',on='product_category_name').drop_duplicates().reset_index(drop=True)
    merged_products_info.drop(columns={'product_category_name'},inplace=True)
    merged_products_info.rename(columns={'product_category_name_english':'product_category'},inplace=True)
    merged_products_info['product_category']=merged_products_info['product_category'].str.strip().str.title()
    merged_products_info['product_id']=merged_products_info['product_id'].str.strip()
    return merged_products_info

products_information= merging_product_info(products_dataset,product_category_name_translation)
products_information['product_category']= products_information['product_category'].fillna('Catgegory_Unavailable')
products_information

,product_id,product_category
0,1e9e8ef04dbcff4541ed26657ea517e5,Perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,Art
2,96bd76ec8810374ed1b65e291975717f,Sports_Leisure
3,cef67bcfe19066a932b7673e239eb23d,Baby
4,9dc1a7de274444849c219cff195d0b71,Housewares
...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,Furniture_Decor
32947,bf4538d88321d0fd4412a93c974510e6,Construction_Tools_Lights
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,Bed_Bath_Table
32949,83808703fc0706a22e264b9d75f04a2e,Computers_Accessories


In [52]:
########################### Cleaning the Sellers Information Dataset #######################################################################
def sellers_dataset_cleaning(df7):
    df7[df7.columns]= df7.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)
    df7['seller_city']= df7['seller_city'].str.strip().str.title()
    duplicated_sellers_dataset= df7[df7.duplicated(subset=['seller_id'],keep=False)]
    df7= df7.drop_duplicates(subset=['seller_id'], keep='first')
    print(f"The total number of duplicated records in the dataset are {duplicated_sellers_dataset.shape[0]}")
    print(f"The total number of records after dropping the duplicates is {df7.shape[0]}")
    return df7, duplicated_sellers_dataset

sellers_dataset, duplicated_sellers_records= sellers_dataset_cleaning(sellers_dataset)

The total number of duplicated records in the dataset are 0
The total number of records after dropping the duplicates is 3095


In [53]:
### EXPORTING THE DATASET TO THE SNOWFLAKE DATABASE FOR ANALYSIS

from sqlalchemy import create_engine
user = ""
password = ""
account = ""
warehouse = ""
database = ""
schema = ""
role = ""   # optional, remove from string if not needed

try:
    engine = create_engine(f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}&role={role}')
    print("The connection string is successfully established")

except Exception as e1:
    print("The connection wasn't established, please check your credentials again")
    print("Error details:", e1)

The connection string is successfully established


In [54]:
### Pushing the cleaned datasets to the snowflake databse
customers_dataset.to_sql("customers_info",con=engine,if_exists='replace',index=False, chunksize=10000)
final_orders.to_sql("orders_info",con=engine, if_exists='replace',index=False, chunksize=10000)
order_items_dataset.to_sql("order_items_info",con=engine, if_exists='replace',index=False, chunksize=10000)
order_reviews_dataset.to_sql("order_reviews_info",con=engine, if_exists='replace',index=False,chunksize=10000)
products_information.to_sql("products_info",con=engine, if_exists='replace',index=False,chunksize=10000)
sellers_dataset.to_sql("sellers_info",con=engine, if_exists='replace',index=False,chunksize=10000)###

3095

In [55]:
############### EXPLORATORY DATA ANALYSIS #############################

In [56]:
print(order_items_dataset.shape[0])
print(order_items_dataset[['order_id','product_id','seller_id']].nunique())

112650
order_id      98666
product_id    32951
seller_id      3095
dtype: int64
